In [28]:
import numpy as np
import pandas 
import math 

In [29]:
#1. Загрузите данные из файла data-logistic.csv. 
data = pandas.read_csv('data-logistic.csv',header=None)
data[:2]

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025


In [30]:
# Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.
y= data.loc[:,0]
y[:2]

0   -1
1    1
Name: 0, dtype: int64

In [41]:
#Здесь xi1 и xi2 — значение первого и второго признаков соответственно на объекте xi. 
x1= data.loc[:,1]
x1[:2]

0   -0.663827
1    1.994596
Name: 1, dtype: float64

In [42]:
x2= data.loc[:,2]
x2[:2]

0   -0.138526
1    2.468025
Name: 2, dtype: float64

In [43]:
#2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. 
# Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [44]:
# Gradient     : w^{t+1} = w^{t} - h * d(Q(w))/dw = w^{t} - h * d(Sum^l_i(q_i(w))/dw
# Stochastic Gr: w^{t+1} = w^{t} - h * d(q_i(w))/dw - q_i - random number from sum
# -> in functionals w1 & w2 from task we have sums, therefore - gradient, not stochastic

In [45]:
def grad_step1(w1,w2,k,C,x1,x2,y):
    l=len(y)
    sum_q=0
    for i in range(1,l):
        sum_q+=y[i]*x1[i]*(1- 1/(1+math.exp(-y[i]*(w1*x1[i]+w2*x2[i]))) )
    return w1+k*sum_q/l - k*C*w1

In [46]:
def grad_step2(w1,w2,k,C,x1,x2,y):
    l=len(y)
    sum_q=0
    for i in range(1,l):
        sum_q+=y[i]*x2[i]*(1- 1/(1+math.exp(-y[i]*(w1*x1[i]+w2*x2[i]))) )
    return w2+k*sum_q/l - k*C*w2

In [48]:
# Q = 1/l sum L(w) =1/l sum log(1 +exp( -y_i(w1*x_i1+w2*x_i2))) +0.5*C|w^2|
# minimize Q wrt w1 and w2
# Градиентный шаг для весов будет заключаться в одновременном обновлении весов w1 и w2 

In [54]:
#3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) 
# логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

def gradient_descent(y,x1,x2,w1=0,w2=0,C=0,k=0.1,precision=1e-5,N_max=10000):
    w1_plus1=w1
    w2_plus1=w2
    #precision=1/N_max
    #Останавливать итерационный процесс можно, например, при близости градиента к нулю 
    #или при слишком малом изменении вектора весов на последней итерации. 
    for i in range(0,N_max):
        w1_plus1=grad_step1(w1,w2,k,C,x1,x2,y)
        w2_plus1=grad_step2(w1,w2,k,C,x1,x2,y)
        previous_step_size = math.sqrt((w1_plus1-w1)**2+(w2_plus1-w2)**2)
        if previous_step_size<precision:
            print("The local minimum occurs at", i)
            break
        else:
            w1=w1_plus1
            w2=w1_plus2
    return [w1_plus1, w2_plus1]


In [55]:
#4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних 
# итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.
